# Prediction

- GROUP STAGE PREDICTION
- KNOCKOUT STAGE PREDICTION

In [83]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import joblib
import warnings
warnings.filterwarnings('ignore')

In [ ]:
last_team_scores = pd.read_csv('data/last_team_scores.csv')
last_team_scores.tail()

## Group Matches

In [ ]:
group_matches = pd.read_csv('data/Qatar_group_stage.csv')
round_16 = group_matches.iloc[48:56, :]
quarter_finals = group_matches.iloc[56:60, :]
semi_finals = group_matches.iloc[60:62, :]
final = group_matches.iloc[62:63, :]
second_final = group_matches.iloc[63:64, :]
group_matches = group_matches.iloc[:48, :]
group_matches.tail()

In [125]:
gs_model = joblib.load("groups_stage_prediction.pkl")

In [87]:
gs_model1 = joblib.load("groups_stage_prediction1.pkl")

In [126]:
ks_model = joblib.load("knockout_stage_prediction.pkl")

In [88]:
ks_model1 = joblib.load("knockout_stage_prediction1.pkl")

## Team Group

In [127]:
team_group = group_matches.drop(['country2'], axis=1)
team_group = team_group.drop_duplicates().reset_index(drop=True)
team_group = team_group.rename(columns = {"country1":"team"})
team_group.head(5)

,team,group
0,Qatar,a
1,Senegal,a
2,England,b
3,USA,b
4,France,d


### Creating functions for prediction

In [128]:
def matches(g_matches):
    
    g_matches.insert(4, 'rank1', g_matches['country1'].map(last_team_scores.set_index('team')['rank']))
    g_matches.insert(5, 'rank2', g_matches['country2'].map(last_team_scores.set_index('team')['rank']))
    pred_set = []

    for index, row in g_matches.iterrows():
        if row['potential1'] > row['potential2'] and abs(row['potential1'] - row['potential2']) > 2:          
            pred_set.append({'Team1': row['country1'], 'Team2': row['country2']})
        elif row['potential2'] > row['potential1'] and abs(row['potential2'] - row['potential1']) > 2:
            pred_set.append({'Team1': row['country2'], 'Team2': row['country1']})
        else:
            if row['rank1'] > row['rank2']:
                pred_set.append({'Team1': row['country1'], 'Team2': row['country2']})
            else:
                pred_set.append({'Team1': row['country2'], 'Team2': row['country1']})
   
    pred_set = pd.DataFrame(pred_set)
    pred_set.insert(2, 'Team1_FIFA_RANK', pred_set['Team1'].map(last_team_scores.set_index('team')['rank']))
    pred_set.insert(3, 'Team2_FIFA_RANK', pred_set['Team2'].map(last_team_scores.set_index('team')['rank']))
    pred_set.insert(4, 'Team1_Goalkeeper_Score', pred_set['Team1'].map(last_team_scores.set_index('team')['goalkeeper_score']))
    pred_set.insert(5, 'Team2_Goalkeeper_Score', pred_set['Team2'].map(last_team_scores.set_index('team')['goalkeeper_score']))
    pred_set.insert(6, 'Team1_Defense', pred_set['Team1'].map(last_team_scores.set_index('team')['defense_score']))
    pred_set.insert(7, 'Team1_Offense', pred_set['Team1'].map(last_team_scores.set_index('team')['offense_score']))
    pred_set.insert(8, 'Team1_Midfield', pred_set['Team1'].map(last_team_scores.set_index('team')['midfield_score']))
    pred_set.insert(9, 'Team2_Defense', pred_set['Team2'].map(last_team_scores.set_index('team')['defense_score']))
    pred_set.insert(10, 'Team2_Offense', pred_set['Team2'].map(last_team_scores.set_index('team')['offense_score']))
    pred_set.insert(11, 'Team2_Midfield', pred_set['Team2'].map(last_team_scores.set_index('team')['midfield_score']))
    return pred_set

In [129]:
def print_results(dataset, y_pred, matches, proba):
  results = []
  for i in range(dataset.shape[0]):
      print()
      if y_pred[i] == 2:
          print(matches.iloc[i, 0] + " vs. " + matches.iloc[i, 1] + " => Draw")
          results.append({'result': 'Draw'})
      elif y_pred[i] == 1:
          print(matches.iloc[i, 0] + " vs. " + matches.iloc[i, 1] + " => Winner: " + dataset.iloc[i, 0])
          results.append({'result': dataset.iloc[i, 0]})
      else:
          print(matches.iloc[i, 0] + " vs. " + matches.iloc[i, 1] + " => Winner: " + dataset.iloc[i, 1])
          results.append({'result': dataset.iloc[i, 1]})
      try:
          print('Probability of ' + dataset.iloc[i, 0] + ' winning: ', '%.3f'%(proba[i][1]))
          print('Probability of Draw: ', '%.3f'%(proba[i][2]))
          print('Probability of ' + dataset.iloc[i, 1] + ' winning: ', '%.3f'%(proba[i][0]))
      except:   
          print('Probability of ' + dataset.iloc[i, 1] + ' winning: ', '%.3f'%(proba[i][0]))   
      print("")
  results = pd.DataFrame(results)
  matches = pd.concat([matches.group, results], axis=1)
  return matches

In [130]:
def winner_to_match(round, prev_match):
    round.insert(0, 'c1', round['country1'].map(prev_match.set_index('group')['result']))
    round.insert(1, 'c2', round['country2'].map(prev_match.set_index('group')['result']))
    round = round.drop(['country1','country2'], axis=1)
    round = round.rename(columns={'c1':'country1', 'c2':'country2'}).reset_index(drop=True)
    return round

In [131]:
def prediction_knockout(round):
    dataset_round = matches(round)
    prediction_round = ks_model.predict(dataset_round)
    proba_round = ks_model.predict_proba(dataset_round)
    results_round = print_results(dataset_round, prediction_round, round, proba_round)
    return results_round

## Gataset Groups

In [132]:
dataset_groups = matches(group_matches)
dataset_groups.tail()

,Team1,Team2,Team1_FIFA_RANK,Team2_FIFA_RANK,Team1_Goalkeeper_Score,Team2_Goalkeeper_Score,Team1_Defense,Team1_Offense,Team1_Midfield,Team2_Defense,Team2_Offense,Team2_Midfield
43,Germany,Costa Rica,12,31,90.0,88.0,84.0,83.0,88.0,72.0,70.0,69.0
44,Uruguay,Ghana,13,60,80.0,74.0,81.0,84.0,80.0,76.0,76.0,78.0
45,Portugal,Korea Republic,8,29,82.0,75.0,85.0,86.0,84.0,73.0,80.0,74.0
46,Serbia,Switzerland,25,14,80.0,85.0,76.0,80.0,82.0,78.0,77.0,80.0
47,Brazil,Cameroon,1,37,89.0,67.0,85.0,86.0,86.0,77.0,78.0,75.0


# GROUP STAGE PREDICTION

In [133]:
prediction_groups = gs_model.predict(dataset_groups)
proba = gs_model.predict_proba(dataset_groups)
results = print_results(dataset_groups, prediction_groups, group_matches, proba)


Qatar vs. Ecuador => Winner: Qatar
Probability of Qatar winning:  0.499
Probability of Draw:  0.283
Probability of Ecuador winning:  0.218


Senegal vs. Netherlands => Winner: Netherlands
Probability of Netherlands winning:  0.486
Probability of Draw:  0.286
Probability of Senegal winning:  0.228


England vs. IR Iran => Winner: England
Probability of England winning:  0.728
Probability of Draw:  0.173
Probability of IR Iran winning:  0.099


USA vs. Wales => Winner: USA
Probability of USA winning:  0.486
Probability of Draw:  0.259
Probability of Wales winning:  0.254


France vs. Australia => Winner: France
Probability of France winning:  0.759
Probability of Draw:  0.146
Probability of Australia winning:  0.095


Denmark vs. Tunisia => Winner: Denmark
Probability of Denmark winning:  0.589
Probability of Draw:  0.234
Probability of Tunisia winning:  0.177


Mexico vs. Poland => Winner: Poland
Probability of Poland winning:  0.393
Probability of Draw:  0.288
Probability of Mexico wi

In [95]:
prediction_groups = gs_model1.predict(dataset_groups)
proba = gs_model1.predict_proba(dataset_groups)
results = print_results(dataset_groups, prediction_groups, group_matches, proba)


Qatar vs. Ecuador => Winner: Qatar
Probability of Qatar winning:  0.489
Probability of Draw:  0.276
Probability of Ecuador winning:  0.235


Senegal vs. Netherlands => Winner: Netherlands
Probability of Netherlands winning:  0.507
Probability of Draw:  0.266
Probability of Senegal winning:  0.227


England vs. IR Iran => Winner: England
Probability of England winning:  0.699
Probability of Draw:  0.178
Probability of IR Iran winning:  0.123


USA vs. Wales => Winner: USA
Probability of USA winning:  0.538
Probability of Draw:  0.255
Probability of Wales winning:  0.207


France vs. Australia => Winner: France
Probability of France winning:  0.748
Probability of Draw:  0.151
Probability of Australia winning:  0.102


Denmark vs. Tunisia => Winner: Denmark
Probability of Denmark winning:  0.609
Probability of Draw:  0.238
Probability of Tunisia winning:  0.152


Mexico vs. Poland => Winner: Poland
Probability of Poland winning:  0.397
Probability of Draw:  0.269
Probability of Mexico wi

## Point tables

In [134]:
team_group['points'] = 0
team_group
for i in range(results.shape[0]):
    for j in range(team_group.shape[0]):
        if results.iloc[i, 1] == team_group.iloc[j, 0]:
            team_group.iloc[j, 2] += 3

In [135]:
print(team_group.groupby(['group','team']).mean().astype(int))

                      points
group team                  
a     Ecuador              0
      Netherlands          9
      Qatar                3
      Senegal              6
b     England              9
      IR Iran              0
      USA                  6
      Wales                3
c     Argentina            9
      Mexico               3
      Poland               6
      Saudi Arabia         0
d     Australia            3
      Denmark              6
      France               9
      Tunisia              0
e     Costa Rica           0
      Germany              9
      Japan                3
      Spain                6
f     Belgium              9
      Canada               0
      Croatia              6
      Morocco              3
g     Brazil               9
      Cameroon             0
      Serbia               6
      Switzerland          3
h     Ghana                3
      Korea Republic       0
      Portugal             9
      Uruguay              6


# Knockout STAGE PREDICTION

**Round of 16**

In [136]:
round_of_16 = team_group[team_group['points'] > 5].reset_index(drop=True)
round_of_16['group'] = (4 - 1/3 * round_of_16.points).astype(int).astype(str) + round_of_16.group 
round_of_16 = round_of_16.rename(columns = {"team":"result"})
    
round_16 = winner_to_match(round_16, round_of_16)
results_round_16 = prediction_knockout(round_16)


Netherlands vs. USA => Winner: Netherlands
Probability of Netherlands winning:  0.763
Probability of USA winning:  0.237


Argentina vs. Denmark => Winner: Argentina
Probability of Argentina winning:  0.699
Probability of Denmark winning:  0.301


Germany vs. Croatia => Winner: Germany
Probability of Germany winning:  0.657
Probability of Croatia winning:  0.343


Brazil vs. Uruguay => Winner: Brazil
Probability of Brazil winning:  0.852
Probability of Uruguay winning:  0.148


England vs. Senegal => Winner: England
Probability of England winning:  0.728
Probability of Senegal winning:  0.272


France vs. Poland => Winner: France
Probability of France winning:  0.831
Probability of Poland winning:  0.169


Belgium vs. Spain => Winner: Spain
Probability of Spain winning:  0.669
Probability of Belgium winning:  0.331


Portugal vs. Serbia => Winner: Portugal
Probability of Portugal winning:  0.663
Probability of Serbia winning:  0.337



In [141]:
results_round_16

,group,result
0,s1,Netherlands
1,s2,Argentina
2,t1,Germany
3,t2,Brazil
4,u1,England
5,u2,France
6,v1,Spain
7,v2,Portugal


**Quarterfinals**

In [137]:
quarter_finals = winner_to_match(quarter_finals, results_round_16)
results_quarter_finals = prediction_knockout(quarter_finals)


Netherlands vs. Argentina => Winner: Argentina
Probability of Netherlands winning:  0.327
Probability of Argentina winning:  0.673


Germany vs. Brazil => Winner: Germany
Probability of Germany winning:  0.531
Probability of Brazil winning:  0.469


England vs. France => Winner: France
Probability of England winning:  0.409
Probability of France winning:  0.591


Spain vs. Portugal => Winner: Portugal
Probability of Portugal winning:  0.558
Probability of Spain winning:  0.442



**Semi-final**

In [138]:
semi_finals = winner_to_match(semi_finals, results_quarter_finals)
results_finals = prediction_knockout(semi_finals)


Argentina vs. Germany => Winner: Argentina
Probability of Germany winning:  0.427
Probability of Argentina winning:  0.573


France vs. Portugal => Winner: France
Probability of Portugal winning:  0.378
Probability of France winning:  0.622



**Final**

In [139]:
final = winner_to_match(final, results_finals)
winner = prediction_knockout(final)


Argentina vs. France => Winner: France
Probability of France winning:  0.592
Probability of Argentina winning:  0.408

